# Baichuan-13B IEPile 测试

In [1]:
import os
os.environ["http_proxy"] = "http://10.6.0.17:8888"
os.environ["https_proxy"] = "http://10.6.0.17:8888"

In [2]:
import torch
from transformers import (
    AutoConfig,
    AutoTokenizer,
    AutoModelForCausalLM,
    GenerationConfig
)
from peft import PeftModel
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_path = 'baichuan-inc/Baichuan2-13B-Chat'
lora_path = 'zjunlp/baichuan2-13b-iepile-lora'
config = AutoConfig.from_pretrained(model_path, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)


/usr/local/lib/python3.9/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib'), PosixPath('/usr/local/nvidia/lib64')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('https'), PosixPath('//pypi.tuna.tsinghua.edu.cn/simple')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//matplotlib_inline.backend_inli

In [3]:
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    config=config,
    device_map="auto",  
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
)


model = PeftModel.from_pretrained(
    model,
    lora_path,
)
model.eval()

WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.4.0+cu121 with CUDA 1201 (you have 2.0.1+cu117)
    Python  3.9.19 (you have 3.9.19)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
Loading checkpoint shards: 100%|██████████| 3/3 [00:59<00:00, 19.68s/it]


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): BaichuanForCausalLM(
      (model): BaichuanModel(
        (embed_tokens): Embedding(125696, 5120, padding_idx=0)
        (layers): ModuleList(
          (0-39): 40 x BaichuanLayer(
            (self_attn): BaichuanAttention(
              (W_pack): Linear(
                in_features=5120, out_features=15360, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=5120, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=15360, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (o_proj): Linear(
                in_features=5120, out_features=5120, bias=Fals

In [4]:
instruction_mapper = {
    'NERzh': "你是专门进行实体抽取的专家。请从input中抽取出符合schema定义的实体，不存在的实体类型返回空列表。请按照JSON字符串的格式回答。",
    'REzh': "你是专门进行关系抽取的专家。请从input中抽取出符合schema定义的关系三元组，不存在的关系返回空列表。请按照JSON字符串的格式回答。",
    'EEzh': "你是专门进行事件提取的专家。请从input中抽取出符合schema定义的事件，不存在的事件返回空列表，不存在的论元返回NAN，如果论元存在多值请返回列表。请按照JSON字符串的格式回答。",
    'EETzh': "你是专门进行事件提取的专家。请从input中抽取出符合schema定义的事件类型及事件触发词，不存在的事件返回空列表。请按照JSON字符串的格式回答。",
    'EEAzh': "你是专门进行事件论元提取的专家。请从input中抽取出符合schema定义的事件论元及论元角色，不存在的论元返回NAN或空字典，如果论元存在多值请返回列表。请按照JSON字符串的格式回答。",
        
    'NERen': "You are an expert in named entity recognition. Please extract entities that match the schema definition from the input. Return an empty list if the entity type does not exist. Please respond in the format of a JSON string.",
    'REen': "You are an expert in relationship extraction. Please extract relationship triples that match the schema definition from the input. Return an empty list for relationships that do not exist. Please respond in the format of a JSON string.",
    'EEen': "You are an expert in event extraction. Please extract events from the input that conform to the schema definition. Return an empty list for events that do not exist, and return NAN for arguments that do not exist. If an argument has multiple values, please return a list. Respond in the format of a JSON string.",
    'EETen': "You are an expert in event extraction. Please extract event types and event trigger words from the input that conform to the schema definition. Return an empty list for non-existent events. Please respond in the format of a JSON string.",
    'EEAen': "You are an expert in event argument extraction. Please extract event arguments and their roles from the input that conform to the schema definition, which already includes event trigger words. If an argument does not exist, return NAN or an empty dictionary. Please respond in the format of a JSON string.", 
}
split_num_mapper = {'NER':6, 'RE':4, 'EE':4, 'EET':4, 'EEA':4}


import json

task = 'NER'
language = 'zh'
schema = ['时间', '人物', '国家', '装备名称', '装备类型', '装备数量', '组织机构', '地点', '任务']
split_num = split_num_mapper[task]
split_schemas = [schema[i:i+split_num] for i in range(0, len(schema), split_num)]
input = '美国海军高级官员透露，随着军队重心向亚洲转移，他们将继续减少在康涅狄格州格罗顿海军基地部署的潜艇数量。根据海军上将格林纳特的预计，到2020年，该海军潜艇基地仅会部署2个中队，每个中队只有6艘攻击型核潜艇，而现在每个中队则拥有16艘潜艇。'
sintructs = []
for split_schema in split_schemas:
    sintruct = json.dumps({'instruction':instruction_mapper[task+language], 'schema':split_schema, 'input':input}, ensure_ascii=False)
    sintructs.append(sintruct)

# sintruct = "{\"instruction\": \"You are an expert in named entity recognition. Please extract entities that match the schema definition from the input. Return an empty list if the entity type does not exist. Please respond in the format of a JSON string.\", \"schema\": [\"person\", \"organization\", \"else\", \"location\"], \"input\": \"284 Robert Allenby ( Australia ) 69 71 71 73 , Miguel Angel Martin ( Spain ) 75 70 71 68 ( Allenby won at first play-off hole )\"}"
# sintruct = "{\"instruction\": \"你是一个命名实体识别专家。请从input中抽取符合schema描述的实体，如果实体类型不存在就返回空列表，并输出为可解析的json格式。\", \"schema\": [\"时间\", \"人物\", \"国家\", \"装备名称\", \"装备类型\", \"装备数量\", \"组织机构\", \"地点\", \"任务\"], \"input\": \"美国海军高级官员透露，随着军队重心向亚洲转移，他们将继续减少在康涅狄格州格罗顿海军基地部署的潜艇数量。根据海军上将格林纳特的预计，到2020年，该海军潜艇基地仅会部署2个中队，每个中队只有6艘攻击型核潜艇，而现在每个中队则拥有16艘潜艇。\"}"
# sintruct = "{\"instruction\": \"你是一个命名实体识别专家。请从input中抽取符合schema描述的实体，如果实体类型不存在就返回空列表，并输出为可解析的json格式。\", \"schema\": {\"时间\": \"实体类型描述事件的发生时间，具体可能是年份或者是可以体现出事情发生时间的表达，例如'2020年'，'9月23日'，'2013年春季'，'今晨'\", \"人物\": \"指任何具体的人名，例如'海军上将格林纳特'。\", \"国家\": \"实体类型指某个具体国家的名字，例如'美国'，'中国'，'俄罗斯'。\", \"装备名称\": \"实体类型描述任何军事装备的具体名称，例如'攻击型核潜艇'。\", \"装备类型\": \"实体类型描述任何军事装备的类型，例如'潜艇'。\", \"装备数量\": \"实体类型描述军事装备的具体数量，例如'2个中队，每个中队6艘'。\", \"组织机构\": \"组织机构实体是指集体性质的组织，比如公司、商铺、俱乐部、学校等。它们在社会和经济活动中扮演一定角色，并拥有一定的人格权，例如'美国海军'，'俄罗斯海军'。\", \"地点\": \"地点实体指具有地理位置信息的实体，它可以代表一个国家、城市、区域、街道等具体的地方或者一个抽象的地理区域。例如'康涅狄格州格罗顿海军基地'。\", \"任务\": \"实体类型描述特定的任务目标，表示某种行为的目的，例如'军队重心向亚洲转移'。\" , }, \"input\": \"美国海军高级官员透露，随着军队重心向亚洲转移，他们将继续减少在康涅狄格州格罗顿海军基地部署的潜艇数量。根据海军上将格林纳特的预计，到2020年，该海军潜艇基地仅会部署2个中队，每个中队只有6艘攻击型核潜艇，而现在每个中队则拥有16艘潜艇。\"}"
sintruct = json.dumps(
    {
        "instruction": "你是一个命名实体识别专家。请从input中抽取符合schema描述的实体，如果实体类型不存在就返回空列表，并输出为可解析的json格式。",
        "schema": {
            "时间": "请抽取文本中的时间信息，包括但不限于具体日期、年份、月份、时间段、事件发生的时间点或时间范围。确保识别出所有相关的时间表达，如‘2023年7月4日’，‘凌晨3点’，‘第二次世界大战期间’。",
            "人物": "请抽取文本中提到的所有人物信息，包括但不限于军事领导人、士兵、政府官员和其他相关人员。注意包括他们的头衔、职务和姓名。如‘将军约翰·史密斯’或‘士兵杰克’。",
            "国家": "请抽取文本中提到的所有国家信息。包括全名、简称或其他指代形式。如‘美国’，‘中国’，‘法国’。",
            "装备名称": "请抽取文本中提到的所有军事装备的具体名称。如‘M1艾布拉姆斯坦克’，‘F-22猛禽战斗机’，‘AK-47突击步枪’。",
            "装备类型": "请抽取文本中提到的所有军事装备的类型信息。包括但不限于装备的分类、用途和技术特征。如‘主战坦克’，‘战斗机’，‘无人机’。",
            "装备数量": "请抽取文本中提到的所有军事装备的数量信息。包括具体数字、近似数量和模糊描述。如‘50辆坦克’，‘数百架战机’，‘大量武器’。",
            "组织机构": "请抽取文本中提到的所有军事相关的组织和机构名称。包括政府部门、军事单位、国际组织等。如‘国防部’，‘第101空降师’，‘北约’。",
            "地点": "请抽取文本中提到的所有地点信息。包括地理位置、具体地点、军事基地和作战区域。如‘阿富汗’，‘华盛顿特区’，‘福特布拉格’。",
            "任务": "请抽取文本中提到的所有军事任务或行动任务。包括任务名称、任务类型和任务目标。如‘沙漠风暴行动’，‘侦察任务’，‘摧毁敌方指挥中心’。" ,
            "行动": "请抽取文本中提到的所有军事行动相关的信息。包括行动名称、行动类型、行动规模和相关策略。如‘诺曼底登陆’，‘突袭’，‘大规模军事行动’，‘闪电战’。" ,
            "事件": "请抽取文本中提到的所有重大军事事件。包括战争、战役、冲突和其他重要事件。如‘越南战争’，‘滑铁卢战役’，‘克什米尔冲突’，‘冷战’。" ,
        },
        "input": "美国海军高级官员透露，随着军队重心向亚洲转移，他们将继续减少在康涅狄格州格罗顿海军基地部署的潜艇数量。根据海军上将格林纳特的预计，到2020年，该海军潜艇基地仅会部署2个中队，每个中队只有6艘攻击型核潜艇，而现在每个中队则拥有16艘潜艇。"
    },
    ensure_ascii=False
)



sintruct = '<reserved_106>' + sintruct + '<reserved_107>'

input_ids = tokenizer.encode(sintruct, return_tensors="pt").to(device)
input_length = input_ids.size(1)
generation_output = model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_length=512, max_new_tokens=256, return_dict_in_generate=True))
generation_output = generation_output.sequences[0]
generation_output = generation_output[input_length:]
output = tokenizer.decode(generation_output, skip_special_tokens=True)

print(output)


{"时间": ["2020年"], "人物": ["格林纳特"], "国家": ["美国海军", "美国"], "装备名称": [], "装备类型": ["攻击型核潜艇"], "装备数量": ["16艘"], "组织机构": [], "地点": ["康涅狄格州格罗顿海军基地"], "任务": [], "行动": [], "事件": []}


In [19]:
sintruct2 = json.dumps(
    {
        "instruction": "你是一个命名实体识别专家。请从input中抽取符合schema描述的实体，如果实体类型不存在就返回空列表，并输出为可解析的json格式。",
        "schema": {
            "时间": "实体类型描述事件的发生时间，具体可能是年份或者是可以体现出事情发生时间的表达，例如'2020年'，'9月23日'，'2013年春季'，'今晨'",
            "人物": "指任何具体的人名，例如'海军上将格林纳特'。",
            "国家": "实体类型指某个具体国家的名字，例如'美国'，'中国'，'俄罗斯'，'以色列'，'韩国'，'前苏联'。",
            "装备名称": "实体类型描述任何军事装备的具体名称，例如'攻击型核潜艇'。",
            "装备类型": "实体类型描述任何军事装备的类型，例如'潜艇'。",
            "装备数量": "实体类型描述军事装备的具体数量，例如'2个中队，每个中队6艘'。",
            "组织机构": "组织机构实体是指集体性质的组织，比如公司、商铺、俱乐部、学校等。它们在社会和经济活动中扮演一定角色，并拥有一定的人格权，例如'美国海军'，'俄罗斯海军'。",
            "地点": "地点实体指具有地理位置信息的实体，它可以代表一个国家、城市、区域、街道等具体的地方或者一个抽象的地理区域。例如'康涅狄格州格罗顿海军基地'。",
            "任务": "实体类型描述特定的任务目标，表示某种行为的目的，例如'军队重心向亚洲转移'。" 
        },
        "input": "一些消息人士透露，这两张地图中包含有关台湾海军舰艇布署的情况。如果发生战争，丢失这两张地图将使台湾海军在面对敌方时失去战斗力。此外，还有一台导弹艇上的笔记本电脑在去年五月丢失，军方检察官认真对此展开调查。"
    },
    ensure_ascii=False
)



sintruct2 = '<reserved_106>' + sintruct2 + '<reserved_107>'

input_ids = tokenizer.encode(sintruct2, return_tensors="pt").to(device)
input_length = input_ids.size(1)
generation_output = model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_length=512, max_new_tokens=256, return_dict_in_generate=True))
generation_output = generation_output.sequences[0]
generation_output = generation_output[input_length:]
output = tokenizer.decode(generation_output, skip_special_tokens=True)

print(output)

{"时间": ["去年五月"], "人物": [], "国家": [], "装备名称": ["导弹艇"], "装备类型": [], "装备数量": [], "组织机构": ["台湾海军"], "地点": [], "任务": []}


In [20]:
sintruct2 = json.dumps(
    {
        "instruction": "你是一个命名实体识别专家。请从input中抽取符合schema描述的实体，如果实体类型不存在就返回空列表，并输出为可解析的json格式。",
        "schema": {
            "时间": "实体类型描述事件的发生时间，具体可能是年份或者是可以体现出事情发生时间的表达，例如'2020年'，'9月23日'，'2013年春季'，'今晨'",
            "人物": "指任何具体的人名，例如'海军上将格林纳特'。",
            "国家": "实体类型指某个具体国家的名字，例如'美国'，'中国'，'俄罗斯'，'以色列'，'韩国'，'前苏联'。",
            "装备名称": "实体类型描述任何军事装备的具体名称，例如'攻击型核潜艇'。",
            "装备类型": "实体类型描述任何军事装备的类型，例如'潜艇'。",
            "装备数量": "实体类型描述军事装备的具体数量，例如'2个中队，每个中队6艘'。",
            "组织机构": "组织机构实体是指集体性质的组织，比如公司、商铺、俱乐部、学校等。它们在社会和经济活动中扮演一定角色，并拥有一定的人格权，例如'美国海军'，'俄罗斯海军'。",
            "地点": "地点实体指具有地理位置信息的实体，它可以代表一个国家、城市、区域、街道等具体的地方或者一个抽象的地理区域。例如'康涅狄格州格罗顿海军基地'。",
            "任务": "实体类型描述特定的任务目标，表示某种行为的目的，例如'军队重心向亚洲转移'。" 
        },
        "input": "奥巴马想在未来的核武器削减谈判中引入新的武器分类，并准备与俄罗斯就战术核武器和贮存弹头进行讨论。"
    },
    ensure_ascii=False
)



sintruct2 = '<reserved_106>' + sintruct2 + '<reserved_107>'

input_ids = tokenizer.encode(sintruct2, return_tensors="pt").to(device)
input_length = input_ids.size(1)
generation_output = model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_length=512, max_new_tokens=256, return_dict_in_generate=True))
generation_output = generation_output.sequences[0]
generation_output = generation_output[input_length:]
output = tokenizer.decode(generation_output, skip_special_tokens=True)

print(output)

{"时间": [], "人物": ["奥巴马"], "国家": ["俄罗斯"], "装备名称": [], "装备类型": [], "装备数量": [], "组织机构": [], "地点": [], "任务": []}


In [2]:

sintruct2 = json.dumps(
    {
        "instruction": "你是一个命名实体识别专家。请从input中抽取符合schema描述的实体，如果实体类型不存在就返回空列表，并输出为可解析的json格式。",
        "schema": {
            "时间": "实体类型描述事件的发生时间，具体可能是年份或者是可以体现出事情发生时间的表达，例如'2020年'，'9月23日'，'2013年春季'，'今晨'",
            "人物": "指任何具体的人名，例如'海军上将格林纳特'。",
            "国家": "实体类型指某个具体国家的名字，例如'美国'，'中国'，'俄罗斯'，'以色列'，'韩国'，'前苏联'。",
            "装备名称": "实体类型描述任何军事装备的具体名称，例如'攻击型核潜艇'。",
            "装备类型": "实体类型描述任何军事装备的类型，例如'潜艇'。",
            "装备数量": "实体类型描述军事装备的具体数量，例如'2个中队，每个中队6艘'。",
            "组织机构": "组织机构实体是指集体性质的组织，比如公司、商铺、俱乐部、学校等。它们在社会和经济活动中扮演一定角色，并拥有一定的人格权，例如'美国海军'，'俄罗斯海军'。",
            "地点": "地点实体指具有地理位置信息的实体，它可以代表一个国家、城市、区域、街道等具体的地方或者一个抽象的地理区域。例如'康涅狄格州格罗顿海军基地'。",
            "任务": "实体类型描述特定的任务目标，表示某种行为的目的，例如'军队重心向亚洲转移'。" 
        },
        "input": "美国国务院在9月23日的一份新闻稿中宣布，双边协商委员会第四次会议于9月11日至21日在日内瓦成功举行。该会议主要就本年度与《削减战略武器条约》实施相关的实际问题进行了美俄间的讨论。"
    },
    ensure_ascii=False
)

sintruct2 = '<reserved_106>' + sintruct2 + '<reserved_107>'

input_ids = tokenizer.encode(sintruct2, return_tensors="pt").to(device)
input_length = input_ids.size(1)
generation_output = model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_length=512, max_new_tokens=256, return_dict_in_generate=True))
generation_output = generation_output.sequences[0]
generation_output = generation_output[input_length:]
output = tokenizer.decode(generation_output, skip_special_tokens=True)

print(output)

NameError: name 'json' is not defined

In [5]:
!CUDA_VISIBLE_DEVICES=1 python src/inference.py \
    --stage sft \
    --model_name_or_path 'baichuan-inc/Baichuan2-13B-Chat' \
    --checkpoint_dir 'zjunlp/baichuan2-13b-iepile-lora' \
    --model_name 'baichuan' \
    --template 'baichuan2' \
    --do_predict \
    --input_file 'data2text/iepile-ner-test-transformed-v2.json' \
    --output_file 'results/baichuan2-13B-iepile-ft-test-output.json' \
    --finetuning_type lora \
    --output_dir 'lora/oneke-continue-test' \
    --predict_with_generate \
    --cutoff_len 512 \
    --bf16 \
    --max_new_tokens 300 \
    --bits 4


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda117.so
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib64'), PosixPath('/usr/local/nvidia/lib')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following direct

In [4]:
!python InstructKGC/ie2instruction/eval_func.py \
  --path1 InstructKGC/results/Baichuan-ft_output_v1.json \
  --task NER

{'total': {'总样本数': 200, '错误数': 0, 'P': 24.08, 'R': 23.73, 'F1': 23.9}}
